In [50]:
import time
import numpy as np
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC


opt = webdriver.ChromeOptions()
opt.add_argument("--incognito")
opt.add_argument("disable-infobars")
opt.add_argument("--disable-extensions")
opt.add_argument("--disable-gpu")
opt.add_argument("--disable-dev-shm-usage")
opt.add_argument("--no-sandbox")
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument("--headless=new")

In [51]:
my_file = open("output.txt", "r") 
  
data = my_file.read() 
  
links = data.split("\n")
my_file.close() 

In [52]:
links[0]

'https://www.athome.com/premium-rolling-christmas-tree-storage-bag-7.5/124183059.html?nav=search'

In [53]:
driver = webdriver.Chrome(options=opt)
driver.get(links[0])
wait = WebDriverWait(driver, 20)

In [54]:
wait.until(EC.visibility_of_element_located((By.XPATH, '//div[@class="row my-md-4 gallery-buybox-wrapper"]')))

<selenium.webdriver.remote.webelement.WebElement (session="bfaba770d1fe14f5837bb2d5c25c06d9", element="1994ACD99F59423FBA3E2B66D615152E_element_76")>

In [55]:
page = driver.page_source
soup = BeautifulSoup(page, "lxml")

In [56]:
header = soup.find(class_="pdp-heading-row row")

In [57]:
header.find(class_="product-name").text.strip()

"Premium Rolling Christmas Tree Storage Bag, 7.5'"

In [58]:
rating = header.find(class_="rating-num")#.text#.strip()
if rating is not None:
    rating.text.strip()
else:
    print("Empty")

In [59]:
amt_of_reviews = header.find(class_="rating-count")#.text.strip()
if amt_of_reviews is not None:
    amt_of_reviews.text.strip()
else:
    print("Empty")

In [60]:
main_area = soup.find(class_="row my-md-4 gallery-buybox-wrapper")

In [61]:
#main_area.find(class_="regular-price active-price").text.strip()

In [62]:
cats = soup.find(class_="product-breadcrumb")

In [63]:
categories_area = cats.find(class_="breadcrumb").select("li")

In [64]:
categories_area[-3].text.strip()

'Christmas'

In [65]:
categories_area[-2].text.strip()

'Christmas Storage'

In [66]:
soup.find(class_="product-brand col-12 p-0").text.strip()

''

In [68]:
money_area = main_area.find(class_="adornment-first-pricing")

In [69]:
money_list = money_area.select("div")[1].get("class")

In [70]:
if "regular-price" in money_list:
    print(money_area.find(class_="regular-price active-price").text.strip())
else:
    print(money_area.find(class_="savings-percentage under-100").text.split()[1])
    print(money_area.find(class_="product-ab-price strikethrough-price regular-price").text.split()[0])
    print(money_area.find(class_="active-price sale-price product-ab-price").text.split()[0])

$24.99


In [71]:
round(len(links)/100)

36

In [72]:
len(set(links))

3614

In [73]:
driver.close()

In [74]:
name = []
actual_price = []
sale_price = []
percent_off = []
review_count = []
rating = []
category = []
sub_category = []
brand = []

runs = 0
loop_repeats = round(len(links)/100)
#driver = webdriver.Chrome(options=opt)
#driver.get(links[0])
#wait = WebDriverWait(driver, 20)
for i in range(runs, loop_repeats + 1):
    if i != loop_repeats:
        l = 0
        for j in links[runs*100:runs*100+100]:
            
            driver = webdriver.Chrome(options=opt)
            driver.get(j)
            wait = WebDriverWait(driver, 20)
            
            try:
                wait.until(EC.visibility_of_element_located((By.XPATH, '//div[@class="row my-md-4 gallery-buybox-wrapper"]')))
            except TimeoutException:
                continue
            page = driver.page_source
            soup = BeautifulSoup(page, "lxml")
            
            header = soup.find(class_="pdp-heading-row row")
            name.append(header.find(class_="product-name").text.strip())
            
            
            main_area = soup.find(class_="row my-md-4 gallery-buybox-wrapper")
            money_area = main_area.find(class_="adornment-first-pricing")
            money_list = money_area.select("div")[1].get("class")
            if "regular-price" in money_list:
                actual_price.append(money_area.find(class_="regular-price active-price").text.strip())
                sale_price.append(money_area.find(class_="regular-price active-price").text.strip())
                percent_off.append("0")
            else:
                percent_off.append(money_area.find(class_="savings-percentage").text.split()[1])
                actual_price.append(money_area.find(class_="product-ab-price strikethrough-price regular-price").text.split()[0])
                sale_price.append(money_area.find(class_="active-price sale-price product-ab-price").text.split()[0])
            
            rating_area = header.find(class_="rating-num")
            if rating_area is not None:
                rating.append(rating_area.text.strip())
            else:
                rating.append("0")
                
            review_area = header.find(class_="rating-count")
            if review_area is not None:
                review_count.append(review_area.text.strip())
            else:
                review_count.append("0")
                
            cats = soup.find(class_="product-breadcrumb")
            categories_area = cats.find(class_="breadcrumb").select("li")
            category.append(categories_area[-3].text.strip())
            sub_category.append(categories_area[-2].text.strip())
            brand.append(soup.find(class_="product-brand col-12 p-0").text.strip())
            driver.close()
    else:
        for i in links[runs*100:-1]:
            driver = webdriver.Chrome(options=opt)
            driver.get(i)
            wait = WebDriverWait(driver, 20)
            
            wait.until(EC.visibility_of_element_located((By.XPATH, '//div[@class="row my-md-4 gallery-buybox-wrapper"]')))
            page = driver.page_source
            soup = BeautifulSoup(page, "lxml")
            
            header = soup.find(class_="pdp-heading-row row")
            name.append(header.find(class_="product-name").text.strip())
            
            
            if "regular-price" in money_list:
                actual_price.append(money_area.find(class_="regular-price active-price").text.strip())
                sale_price.append(money_area.find(class_="regular-price active-price").text.strip())
                percent_off.append("0")
            else:
                percent_off.append(money_area.find(class_="savings-percentage").text.split()[1])
                actual_price.append(money_area.find(class_="product-ab-price strikethrough-price regular-price").text.split()[0])
                sale_price.append(money_area.find(class_="active-price sale-price product-ab-price").text.split()[0])
            
            rating_area = header.find(class_="rating-num")#.text#.strip()
            if rating_area is not None:
                rating.append(rating_area.text.strip())
            else:
                rating.append("0")
                
            review_area = header.find(class_="rating-count")#.text.strip()
            if review_area is not None:
                review_count.append(review_area.text.strip())
            else:
                review_count.append("0")
                
            cats = soup.find(class_="product-breadcrumb")
            category.append(categories_area[-3].text.strip())
            sub_category.append(categories_area[-2].text.strip())
            brand.append(soup.find(class_="product-brand col-12 p-0").text.strip())
            
            driver.close()
    
    runs += 1
    print(f"Runs completed: {runs}")

Runs completed: 1
Runs completed: 2
Runs completed: 3
Runs completed: 4
Runs completed: 5
Runs completed: 6
Runs completed: 7
Runs completed: 8
Runs completed: 9
Runs completed: 10
Runs completed: 11
Runs completed: 12
Runs completed: 13
Runs completed: 14
Runs completed: 15
Runs completed: 16
Runs completed: 17
Runs completed: 18
Runs completed: 19
Runs completed: 20
Runs completed: 21
Runs completed: 22
Runs completed: 23
Runs completed: 24
Runs completed: 25
Runs completed: 26
Runs completed: 27
Runs completed: 28
Runs completed: 29
Runs completed: 30
Runs completed: 31
Runs completed: 32
Runs completed: 33
Runs completed: 34
Runs completed: 35
Runs completed: 36
Runs completed: 37


In [75]:
len(name)

3269

In [76]:
len(set(name))

3225

In [77]:
len(actual_price)

3269

In [78]:
len(sale_price)

3269

In [79]:
len(percent_off)

3269

In [80]:
len(review_count)

3269

In [81]:
len(rating)

3269

In [82]:
len(category)

3269

In [83]:
len(sub_category)

3269

In [84]:
len(brand)

3269

In [85]:
data = {"Name": name, "Actual Price": actual_price, "Sale Price": sale_price, "Discount %": percent_off, "Rating": rating, "Number of Reviews": review_count, "Category": category, "Sub-Category": sub_category, "Brand": brand}

In [86]:
df = pd.DataFrame.from_dict(data)
df.head()

,Name,Actual Price,Sale Price,Discount %,Rating,Number of Reviews,Category,Sub-Category,Brand
0,"Premium Rolling Christmas Tree Storage Bag, 7.5'",$24.99,$24.99,0,4.2,96,Christmas,Christmas Storage,
1,"Inflatable Colossal Waving Santa, 20'",$199.99,$99.99,50%,3.2,23,Christmas Inflatables & Outdoor Decor,Oversized Decor,
2,"& Elf Pets® Merry Christmas Table Decor, 6""",$4.99,$2.49,50%,0,0,Christmas,The Elf on the Shelf Collection,The Elf on the Shelf®
3,"Christmas Magic Card Holder Table Decor, 11x7",$9.99,$4.99,50%,5,1,Christmas,The Elf on the Shelf Collection,The Elf on the Shelf®
4,150-Count Silver Ornament Hooks,$1.99,$0.99,50%,4.1,39,Christmas,Christmas Ornaments,


In [87]:
df.to_csv("../Datasets/at_home_christmas_deco.csv")